In [1]:
# Install Packages
# !pip install pymongo
# !pip freeze

In [2]:
# Import Libraries
import pymongo
import json
from bson.objectid import ObjectId
from datetime import datetime
import pandas as pd

In [14]:
#Establish the connection and make the database
client = pymongo.MongoClient("mongodb://localhost:27017/") 
db_list = client.list_database_names()
if "Tweets_DB" in db_list:
    print("Database already exists. Will be dropped...")
    client.drop_database("Tweets_DB")
db = client["Tweets_DB"]
col_list = db.list_collection_names()
if "Tweets_data" in col_list:
        print("Tweets Collection already exists. Will be dropped...")
        db.Tweets_data.drop()
tweets_collec = db["Tweets_data"]
print("Database connection established")

Database already exists. Will be dropped...
Database connection established


In [15]:
# adding collection to the database
tweets_collec.insert_one({"id": 1, "User_Name": "Rohit"})

In [16]:
tweets_collec.count_documents({})

1

In [17]:
client.list_database_names()

['Tweets_DB', 'admin', 'config', 'local']